## Import packages

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt 
%matplotlib inline
import os

## Import Data and DataLoader

In [3]:


transforms_func = transforms.Compose([
    transforms.RandomResizedCrop(416), 
    transforms.RandomHorizontalFlip(), 
    transforms.RandomVerticalFlip(),   
    transforms.RandomRotation(15),     
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), 
    transforms.Resize((416, 416)),      
    transforms.ToTensor(),             
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Load the training dataset with augmentations
train_datasets = datasets.ImageFolder(
    'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets/train',
    transform=transforms_func
)

# Define the validation transformation (typically only resizing and normalization)
val_transforms = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_datasets = datasets.ImageFolder(
    'C:\\Users\\sshak\\Downloads\\Seen Datasets1\\Seen Datasets/val',
    transform=val_transforms
)



data_dir = 'C:\\Users\\sshak\\Downloads\\Seen Datasets\\Seen Datasets'
batch_size = 32
train_dataloader = DataLoader(train_datasets, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_datasets, batch_size=batch_size, shuffle=False)


## Model Building

In [2]:

# Depthwise Separable Convolution
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

# Define MobileNet
class MobileNet(nn.Module):
    def __init__(self, num_classes=25):
        super(MobileNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            DepthwiseSeparableConv(32, 64, stride=1),
            DepthwiseSeparableConv(64, 128, stride=2),
            DepthwiseSeparableConv(128, 128, stride=1),
            DepthwiseSeparableConv(128, 256, stride=2),
            DepthwiseSeparableConv(256, 256, stride=1),
            DepthwiseSeparableConv(256, 512, stride=2),

            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),

            DepthwiseSeparableConv(512, 1024, stride=2),
            DepthwiseSeparableConv(1024, 1024, stride=1),

            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


## Training Pipeline / Loop

In [5]:

# Training function with gradient accumulation
def train_mobilenet(num_classes, epochs, learning_rate, train_dataloader, val_dataloader, model_path, accumulation_steps=4):
    # Initialize model, loss function, and optimizer
    model = MobileNet(num_classes=num_classes).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        train_loader_tqdm = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs}')
        optimizer.zero_grad()
        
        for i, (inputs, labels) in enumerate(train_loader_tqdm):
            inputs, labels = inputs.cuda(), labels.cuda()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            train_loader_tqdm.set_postfix(loss=running_loss/((i+1)/accumulation_steps), accuracy=100*correct/total)
        
        train_loss = running_loss / len(train_dataloader)
        train_accuracy = 100 * correct / total
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%')
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_dataloader, desc=f'Validation Epoch {epoch+1}/{epochs}'):
                inputs, labels = inputs.cuda(), labels.cuda()
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_dataloader)
        val_accuracy = 100 * val_correct / val_total
        
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')
    
    # Save the trained model
    torch.save(model.state_dict(), model_path)

## Hyper parameter and Training

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:

train_mobilenet(
    num_classes=25,
    epochs=10,
    learning_rate=0.001,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    model_path='mobilenet_model.pth',
    accumulation_steps=4  # Accumulate gradients over 4 steps
)


Epoch 1/10: 100%|██████████| 704/704 [08:40<00:00,  1.35it/s, accuracy=12.3, loss=11.4]


Epoch 1/10, Loss: 2.8467, Accuracy: 12.35%


Validation Epoch 1/10: 100%|██████████| 235/235 [00:57<00:00,  4.09it/s]


Validation Loss: 2.7609, Accuracy: 14.99%


Epoch 2/10: 100%|██████████| 704/704 [08:29<00:00,  1.38it/s, accuracy=19.6, loss=10.1]


...

Epoch 6/10, Loss: 1.8465, Accuracy: 40.19%


Validation Epoch 6/10: 100%|██████████| 235/235 [01:11<00:00,  3.28it/s]


Validation Loss: 2.5131, Accuracy: 27.48%


Epoch 7/10:  30%|██▉       | 210/704 [02:48<06:36,  1.25it/s, accuracy=50.2, loss=6.12]


KeyboardInterrupt: 

RE-TRAINING  MODEL 

In [5]:


def train_mobilenet(num_classes, epochs, learning_rate, train_dataloader, val_dataloader, model_path, accumulation_steps=4):
    # Initialize model, loss function, and optimizer
    model = MobileNet(num_classes=25).cuda()
    model.load_state_dict(torch.load('mobilenet_model(1).pth'))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        train_loader_tqdm = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs}')
        optimizer.zero_grad()
        
        for i, (inputs, labels) in enumerate(train_loader_tqdm):
            inputs, labels = inputs.cuda(non_blocking=True), labels.cuda(non_blocking=True)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            train_loader_tqdm.set_postfix(loss=running_loss/((i+1)/accumulation_steps), accuracy=100*correct/total)
        
        train_loss = running_loss / len(train_dataloader)
        train_accuracy = 100 * correct / total
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%')
        
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in tqdm(val_dataloader, desc=f'Validation Epoch {epoch+1}/{epochs}'):
                inputs, labels = inputs.cuda(non_blocking=True), labels.cuda(non_blocking=True)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_dataloader)
        val_accuracy = 100 * val_correct / val_total
        
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')
    
    # Save the trained model
    torch.save(model.state_dict(), model_path)

# Example usage
if __name__ == "__main__":
    # Data transforms
    transforms_func = transforms.Compose([
        transforms.Resize((416, 416)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),   
        
        # transforms.RandomRotation(15),
        # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    val_transforms = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create datasets
    train_datasets = datasets.ImageFolder(
        'C:\\Users\\sshak\\Downloads\\Seen Datasets2\\Seen Datasets/train',
        transform=transforms_func
    )
    val_datasets = datasets.ImageFolder(
        'C:\\Users\\sshak\\Downloads\\Seen Datasets2\\Seen Datasets/val',
        transform=val_transforms
    )

    batch_size = 32
    train_dataloader = DataLoader(train_datasets, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_datasets, batch_size=batch_size, shuffle=False)



In [8]:

train_mobilenet(
        num_classes=25,
        epochs=10,
        learning_rate=0.001,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        model_path='mobilenet_model(2).pth',
        accumulation_steps=4  # Accumulate gradients over 4 steps
    )

Epoch 1/10: 100%|██████████| 704/704 [18:01<00:00,  1.54s/it, accuracy=92.7, loss=0.913]


Epoch 1/10, Loss: 0.2281, Accuracy: 92.73%


Validation Epoch 1/10: 100%|██████████| 235/235 [02:57<00:00,  1.32it/s]


Validation Loss: 0.2475, Accuracy: 92.69%


Epoch 2/10: 100%|██████████| 704/704 [15:00<00:00,  1.28s/it, accuracy=95.2, loss=0.634]


...

Validation Loss: 0.2042, Accuracy: 93.89%


Epoch 10/10: 100%|██████████| 704/704 [15:48<00:00,  1.35s/it, accuracy=97.7, loss=0.284]


Epoch 10/10, Loss: 0.0709, Accuracy: 97.72%


Validation Epoch 10/10: 100%|██████████| 235/235 [03:11<00:00,  1.23it/s]

Validation Loss: 0.2135, Accuracy: 94.11%




USING MODEL


In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
import os
from PIL import Image

# Depthwise Separable Convolution
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

# Define MobileNet
class MobileNet(nn.Module):
    def __init__(self, num_classes=25):
        super(MobileNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            DepthwiseSeparableConv(32, 64, stride=1),
            DepthwiseSeparableConv(64, 128, stride=2),
            DepthwiseSeparableConv(128, 128, stride=1),
            DepthwiseSeparableConv(128, 256, stride=2),
            DepthwiseSeparableConv(256, 256, stride=1),
            DepthwiseSeparableConv(256, 512, stride=2),

            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),

            DepthwiseSeparableConv(512, 1024, stride=2),
            DepthwiseSeparableConv(1024, 1024, stride=1),

            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Define the same transformations as used during training
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

model = MobileNet(num_classes=25).cuda()
model.load_state_dict(torch.load('mobilenet_model.pth'))
model.cuda()
model.eval()  # Set model to evaluation mode

def predict(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Move the image to GPU
    image = image.cuda(non_blocking=True)
    
    # Make prediction
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    return predicted.item()  # Return the predicted class index

# Example usage
import os



image_directory = r"C:\Users\sshak\Downloads\Seen Datasets\Seen Datasets\val\White-Breasted-Kingfisher"

# Loop through every image in the folder
for filename in os.listdir(image_directory):
    # Construct the full path to the image
    image_path = os.path.join(image_directory, filename)
    
    # Make sure it's a file (and optionally check if it's an image)
    if os.path.isfile(image_path):
        predicted_class = predict(image_path)
        print(f'Predicted class: {predicted_class}')



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\sshak\\Downloads\\Seen Datasets\\Seen Datasets\\val\\White-Breasted-Kingfisher'

TESTING MODEL


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

# Depthwise Separable Convolution
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

# Define MobileNet
class MobileNet(nn.Module):
    def __init__(self, num_classes=25):
        super(MobileNet, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            DepthwiseSeparableConv(32, 64, stride=1),
            DepthwiseSeparableConv(64, 128, stride=2),
            DepthwiseSeparableConv(128, 128, stride=1),
            DepthwiseSeparableConv(128, 256, stride=2),
            DepthwiseSeparableConv(256, 256, stride=1),
            DepthwiseSeparableConv(256, 512, stride=2),

            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),
            DepthwiseSeparableConv(512, 512, stride=1),

            DepthwiseSeparableConv(512, 1024, stride=2),
            DepthwiseSeparableConv(1024, 1024, stride=1),

            nn.AdaptiveAvgPool2d(1)
        )
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Define test transformations
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    # # transforms.RandomResizedCrop(416), 
    transforms.RandomHorizontalFlip(), 
    transforms.RandomVerticalFlip(),   
    transforms.RandomRotation(10),     
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load test data
test_dataset = ImageFolder(root=r'C:\Users\sshak\Downloads\Seen Datasets2\Seen Datasets\val', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load the saved model
model = MobileNet(num_classes=25).cuda()
model.load_state_dict(torch.load('model.pth'))
model = model.cuda()



In [ ]:
# !pip install seaborn

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Updated test function to include confusion matrix and classification report
def test_model(model, test_loader, class_names):
    model.eval()
    y_true = []
    y_pred = []
    y_probs = []
    criterion = nn.CrossEntropyLoss()
    total_loss = 0.0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)  # Get probabilities
            _, predicted = torch.max(outputs.data, 1)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_probs.extend(probs.cpu().numpy())  # Collect probabilities
            total_loss += criterion(outputs, labels).item()

    # Convert lists to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_probs = np.array(y_probs)

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    roc_auc = roc_auc_score(y_true, y_probs, multi_class='ovr')
    logloss = log_loss(y_true, y_probs)

    # Print metrics
    print(f'Test Accuracy: {accuracy:.2f}%')
    print(f'Test Precision: {precision:.2f}')
    print(f'Test Recall: {recall:.2f}')
    print(f'Test F1 Score: {f1:.2f}')
    print(f'Test ROC-AUC: {roc_auc:.2f}')
    print(f'Test Log Loss: {logloss:.2f}')
    
    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
    
    # Print classification report for each class
    report = classification_report(y_true, y_pred, target_names=class_names)
    print('Classification Report:')
    print(report)

# Class names corresponding to your 25 classes
class_names = test_dataset.classes

# Run the testing function
test_model(model, test_loader, class_names)
